In [1]:
import numpy as np
from functools import reduce
import itertools

In [2]:
def encode_rle(x):
    length = len(x)

    if length == 0:
        return None
    else:
        next_comp = np.array(x[1:] != x[:-1])
        indices = np.append(np.where(next_comp), length - 1)
        counts = np.diff(np.append(-1, indices))

        return(x[indices], counts)

In [3]:
def find_index_to_insert(indices, shape):
    rows_num_to_insert = shape[0]
    cols_num_to_insert = shape[1]
    rows_encode_rle = encode_rle(indices[0])
    # строки c достаточным количеством столбцов
    rows = rows_encode_rle[0][np.where(rows_encode_rle[1] >= cols_num_to_insert)[0]]
    # теперь надо найти среди них такие, чтобы они шли подряд нужное кол-во раз
    row_ranges = []
    for i in range(len(rows) - rows_num_to_insert + 1):
        row_range = []
        for j in range(i, i + rows_num_to_insert):
            row_range.append(rows[j])
            
        for j in range(len(row_range) - 1):
            if row_range[j] - row_range[j+1] != -1:
                break
        else:
            row_ranges.append(row_range)
    # теперь надо найти в каком из этих ренджей пересечение стобцов в каждой строке равно нужному количеству столбцов
    # столбwы каждой строки
    cols_of_every_row = {}
    for row in rows:
        cols_of_every_row[row] = indices[1][np.where(indices[0] == row)]
        
    # ищем пересчение
    row_ranges_with_intersects = []
    for set_of_rows in row_ranges:
        intersect = reduce(np.intersect1d, (cols_of_every_row[row] for row in set_of_rows))

        if len(intersect) < cols_num_to_insert:
            continue

        for i in range(len(intersect) - cols_num_to_insert + 1):
            intersects_range = []
            
            for j in range(i, i + cols_num_to_insert):
                intersects_range.append(intersect[j])
            
            for j in range(len(intersects_range) - 1):
                if intersects_range[j] - intersects_range[j+1] != -1:
                    break
            else:
                row_ranges_with_intersects.append((set_of_rows, intersects_range))
        
    if len(row_ranges_with_intersects) == 0:
        return ()
    # итого мы получаем по идеи все индексы куда можно пихнуть матрицу
    # пока вернем первый возможный вариант, потом может чего дооптимизируем, если 
    indices_to_insert = np.transpose(np.array(list(itertools.product(row_ranges_with_intersects[0][0], row_ranges_with_intersects[0][1]))))
    return indices_to_insert[0], indices_to_insert[1]
    

In [4]:
def main(A_size, matrices_count):
    A = np.zeros(A_size, dtype='int')
    print(A)
    matrices = []
    for i in range(matrices_count):
        n = np.random.randint(1, A_size[0])
        m = np.random.randint(1, A_size[1])
        Z = np.array([np.random.randint(1, 10)] * (n*m)).reshape((n, m))
        matrices.append({'matrix': Z, 'matrix_sum': Z.sum()})
        
    X = np.array([np.random.randint(1, 10)] * (2*1)).reshape((2, 1))
    matrices.append({'matrix': X, 'matrix_sum': X.sum()})
    matrices = sorted(matrices, reverse=True, key=lambda item: item['matrix_sum'])
    for _ in matrices:
        print(_['matrix'])
    
    for item in matrices:
        empty_indices = np.where(A == 0)
        indices_to_insert = find_index_to_insert(empty_indices, item['matrix'].shape)

        if len(indices_to_insert) == 0:
            continue
            
        A[indices_to_insert] = item['matrix'].reshape(1,-1)
        
    print(A)
    print('Sum', A.sum())
    print('---------------------------------------------------------------------------------------------')
    min_axis_1 = np.amin(A, axis=1)
    indices_min_axis_1 = np.where(A == min_axis_1[:,None])
    max_of_mins_axis_1 = np.max(min_axis_1)
    indices_of_max_of_mins_axis_1 = np.where((A == min_axis_1[:,None]) & (A == max_of_mins_axis_1))
    print('Mins axis=1', min_axis_1)
    print('Indices of mins axis=1', indices_min_axis_1)
    print('Max of mins axis=1', max_of_mins_axis_1)
    print('Indices of max of mins axis=1', indices_of_max_of_mins_axis_1)
    print('---------------------------------------------------------------------------------------------')
    min_axis_0 = np.amin(A, axis=0)
    indices_min_axis_0 = np.where(A == min_axis_0[None,:])
    max_of_mins_axis_0 = np.max(min_axis_0)
    indices_of_max_of_mins_axis_0 = np.where((A == min_axis_0[None,:]) & (A == max_of_mins_axis_0))
    print('Mins axis=0', min_axis_0)
    print('Indices of mins axis=0', indices_min_axis_0)
    print('Max of mins axis=0', max_of_mins_axis_0)
    print('Indices of max of mins axis=0', indices_of_max_of_mins_axis_0)

In [8]:
main((7, 7), 7)

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[[3 3 3 3 3 3]
 [3 3 3 3 3 3]
 [3 3 3 3 3 3]
 [3 3 3 3 3 3]
 [3 3 3 3 3 3]
 [3 3 3 3 3 3]]
[[6 6 6 6]
 [6 6 6 6]]
[[4 4 4 4 4]
 [4 4 4 4 4]]
[[3 3 3 3]
 [3 3 3 3]
 [3 3 3 3]]
[[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
[[8]
 [8]]
[[3]
 [3]
 [3]
 [3]]
[[2]
 [2]]
[[3 3 3 3 3 3 8]
 [3 3 3 3 3 3 8]
 [3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3]
 [3 3 3 3 3 3 3]
 [0 0 0 0 0 0 0]]
Sum 136
---------------------------------------------------------------------------------------------
Mins axis=1 [3 3 3 3 3 3 0]
Indices of mins axis=1 (array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
       3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6,
       6, 6, 6]), array([0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5